# Tech Challenge Fase 3 - EDA
## Pós-graduação em Machine Learning
**Autor**: Alan Alves  
**Data**: 26/10/2025

In [ ]:
# Carrega Módulos
import pandas as pd
from pathlib import Path
from datetime import datetime

In [26]:
# Configurações
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
print(f"Endereço raiz: {PROJECT_ROOT}")

Endereço raiz: /Users/alan/Data Science Projects/TechChallenge3


### Inspeção Inicial

In [ ]:
# Load data
airlines_path = DATA_DIR / "airlines.csv"
airport_path = DATA_DIR / "airports.csv"
flights_path = DATA_DIR / "flights.csv"

start_time = datetime.now()
airlines_df = pd.read_csv(airlines_path)
airport_df = pd.read_csv(airport_path)
flights_df = pd.read_csv(flights_path)
end_time = datetime.now()

print(f"Dados carregados com sucesso em {((end_time-start_time).seconds)} seconds.")


/var/folders/6x/tz4_svyx1mv9mc68z3jl0_s00000gn/T/ipykernel_2808/3452739918.py:10: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv(flights_path)


Dados carregados com sucesso em 23 seconds.
